In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('./'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

The goal is to create an algorithm that can play the game “Tetris” for as long as possible.


# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

There is no data to retrieve and represent easily. \
Getting the data here will contain letting the population of models play the game and evaluating their performance at each stage. The model will take in the board state and the piece state and make a decision based on that.

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

The Tetris game is provided for us.
* It consists of a board with a dimensions of 10 by 20.
* Each turn of the game, the board generates a randomly selected piece and the player has to select a place to drop the piece.
* The pieces consist of an l piece, o piece, L piece, J piece, z piece, s piece, and t piece. (7 in total).
* The player is allowed to rotate the pieces 0, 90, 180, or 270 degrees.
* Each turn, filled up rows are cleared.
* The goal of this game is to last the longest. As a result, this consists of clearing the most lines, which we will use to measure how well the model does.

There isn’t a way to properly explore any data here, so instead let’s analyze the board and the properties. \
There’s 9 properties of interest: height sum, number of holes, bumpiness, number of pits, largest well size, number of columns with holes, row transitions, column transitions, and number of non empty rows. \
* Height Sum - Sum of the heights in each column
* Number of holes - The number of empty cells on the board surrounded by blocks on top and bottom
* Bumpiness - The difference of column heights between each adjacent column
* Number of pits - Number of columns that are just all empty
* Largest Well SIze - Largest difference between adjacent columns
* Number of Columns with holes - Number of columns with gaps in the middle
* Row Transitions - Counts the number of times adjacent cells change by row.
* Column Transitions - Counts the number of times adjacent cells change by column
* Number of nonempty rows - self-explanatory. 

The given code contains the game, board class, piece class, and several prewritten algorithms \
Models: 
* Genetic
* Greedy
* Monte Carlo
* Random 

The way the model is supposed to perform is that at each step the game, the model is given the board state and the piece. The model mus then output the x value and piece orientation for the move. 

# 4.Prepare the Data


Apply any data transformations and explain what and why


There is no real data preparation that can be done as there isn’t a set database. The data (board and piece states) is already prepared when given to the model. The model just gets the properties stated above and then processes it.

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


After some research, the best way of approaching this is using reinforcement learning or a genetic algorithm. After several long days of trying reinforcement learning, the model didn’t seem to want to improve after dropping 30 pieces. Despite tweaking the rewards function and making sure the code works as intended, I decided to cut my losses and move onto genetic. <br> 
The model provided below simply sets the gene (the weights for the 9 features stated above) and contains a “get_best_move” function to get the best move that the given gene states is the best. <br>

In [ ]:
import numpy as np
from copy import copy, deepcopy
import random
from genetic_helpers import *


class Genetic_AI:
    def __init__(self, genotype=None, num_features=9, mutate=False,  noise_sd=.2):

        if(genotype is None):
            # randomly init genotype [-1, 1]
            self.genotype = np.array([random.uniform(-1, 1) for _ in range(num_features)])
        else:
            if(mutate == False):
                self.genotype = genotype
            else:
                # mutate given genotype
                mutation = np.array([np.random.normal(1, noise_sd) for i in range(num_features)])
                self.genotype = genotype * mutation



    def valuate(self, board):
        # valuates based on the properties
        

        peaks = get_peaks(board)
        highest_peak = np.max(peaks)
        holes = get_holes(peaks, board)
        wells = get_wells(peaks)

        rating_funcs = {
            'agg_height': np.sum(peaks),
            'n_holes': np.sum(holes),
            'bumpiness': get_bumpiness(peaks),
            'num_pits': np.count_nonzero(np.count_nonzero(board, axis=0) == 0),
            'max_wells': np.max(wells),
            'n_cols_with_holes': np.count_nonzero(np.array(holes) > 0),
            'row_transitions': get_row_transition(board, highest_peak),
            'col_transitions': get_col_transition(board, peaks),
            'cleared': np.count_nonzero(np.mean(board, axis=1))
        }

        ratings = np.array([*rating_funcs.values()], dtype=float)
        aggregate_rating = np.dot(ratings, self.genotype)

        return aggregate_rating


    def get_best_move(self, board, piece):

        # Gets the best move based on board, next piece, and genotype
        

        best_x = -1000
        max_value = -1000
        best_piece = None
        for i in range(4):
            piece = piece.get_next_rotation()
            for x in range(board.width):
                try:
                    y = board.drop_height(piece, x)
                except:
                    continue

                board_copy = deepcopy(board.board)
                for pos in piece.body:
                    board_copy[y + pos[1]][x + pos[0]] = True

                np_board = bool_to_np(board_copy)
                score = self.valuate(np_board)

                if score > max_value:
                    max_value = score
                    best_x = x
                    best_piece = piece
        return best_x, best_piece


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


To train the model, we create a population of 100 models with random genes. Then, they play the game and the best 30 get to “reproduce” (combine randomly to form another genotype) and populate for a next iteration. This runs for a set amount of times and the best model is saved and how well it does is logged. The best of these runs will be saved and utilized for the final model.

In [ ]:
from genetic2 import Genetic_AI
from game import Game

from multiprocessing import Pool
import random
import numpy as np
import pickle
import joblib



def run(model):
        game = Game("student", model)
        drops, clears = game.run_no_visual()
        return clears

    
    
def children(parent1, parent2):
    #setting child as one genotype and then randomly selecting items of the other to form child
    child = parent1.genotype
    for i in range(len(parent1.genotype)):
        if random.randint(0, 1) == 1:
            child[i] = parent2.genotype[i]

    return Genetic_AI(genotype=np.array(child), mutate=True)





def main():
    popsize = 100
    generations = 100

    population = []
    for i in range(popsize):
        population.append(Genetic_AI())

    
    bestscore = 0
    bestgen = 0
    bestchild = 0
    bestchildgen = 0
    bestscoregen = 0   
    with Pool(30) as pool:
        for gen in range(generations):
            scores = pool.map(run, population)
            scoreslist = []
            for i, score in enumerate(scores):
                scoreslist.append((score,i))
            scoresort = sorted(scoreslist, key=lambda x: x[0], reverse=True)

            survivors = []
            for a, b in scoresort[:int(0.3*popsize)]:
                survivors.append(population[b]
            

            bestrn, childID = scoresort[0][0], scoresort[0][1]

            if bestrn > bestscore:
                bestscore = bestrn
                bestgen = gen+1
                bestchild = childID
                bestchildgen = population[int(childID)]
                bestscoregen = bestrn


            bestscore = 0
            joblib.dump(bestchildgen, f'genes/test2/g{bestgen}.joblib')
            with open(f'genes/test2/log.txt', 'a') as f:
                f.write(f'Generation: {bestgen}, Child: {bestchild}, Score: {bestscoregen}\n')
                
            
            newpop = []
            counts = 0
            while counts < popsize:
                parent1 = random.choice(survivors)
                parent2 = random.choice(survivors)
                child = children(parent1, parent2)
                if child is not None:
                    newpop.append(child)
                    counts += 1

            population = newpop

        joblib.dump(population[int(childID)], 'genes/test2f.joblib')

    print('Done')

if __name__ == '__main__':
    main()


In the end, generation 17 performed the best with ~14000 lines cleared c:

# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


### Frame the Problem
The goal is to create an algorithm that can play the game “Tetris” for as long as possible.
### Get the Data
There is no data to retrieve and represent easily. \
Getting the data here will contain letting the population of models play the game and evaluating their performance at each stage. The model will take in the board state and the piece state and make a decision based on that.
### Explore the Data
The Tetris game is provided for us.
* It consists of a board with a dimensions of 10 by 20.
* Each turn of the game, the board generates a randomly selected piece and the player has to select a place to drop the piece.
* The pieces consist of an l piece, o piece, L piece, J piece, z piece, s piece, and t piece. (7 in total).
* The player is allowed to rotate the pieces 0, 90, 180, or 270 degrees.
* Each turn, filled up rows are cleared.
* The goal of this game is to last the longest. As a result, this consists of clearing the most lines, which we will use to measure how well the model does.

There isn’t a way to properly explore any data here, so instead let’s analyze the board and the properties. \
There’s 9 properties of interest: height sum, number of holes, bumpiness, number of pits, largest well size, number of columns with holes, row transitions, column transitions, and number of non empty rows. \
* Height Sum - Sum of the heights in each column
* Number of holes - The number of empty cells on the board surrounded by blocks on top and bottom
* Bumpiness - The difference of column heights between each adjacent column
* Number of pits - Number of columns that are just all empty
* Largest Well SIze - Largest difference between adjacent columns
* Number of Columns with holes - Number of columns with gaps in the middle
* Row Transitions - Counts the number of times adjacent cells change by row.
* Column Transitions - Counts the number of times adjacent cells change by column
* Number of nonempty rows - self-explanatory. 

The given code contains the game, board class, piece class, and several prewritten algorithms \
Models: 
* Genetic
* Greedy
* Monte Carlo
* Random 

The way the model is supposed to perform is that at each step the game, the model is given the board state and the piece. The model mus then output the x value and piece orientation for the move. 
### Prepare the Data
There is no real data preparation that can be done as there isn’t a set database. The data (board and piece states) is already prepared when given to the model. The model just gets the properties stated above and then processes it.
### Model the Data
After some research, the best way of approaching this is using reinforcement learning or a genetic algorithm. After several long days of trying reinforcement learning, the model didn’t seem to want to improve after dropping 30 pieces. Despite tweaking the rewards function and making sure the code works as intended, I decided to cut my losses and move onto genetic. <br> 
The model provided below simply sets the gene (the weights for the 9 features stated above) and contains a “get_best_move” function to get the best move that the given gene states is the best. <br>
### Fine Tune the Model
To traini the model, we create a population of 100 models with random genes. Then, they play the game and the best 30 get to “reproduce” (combine randomly to form another genotype) and populate for a next iteration. This runs for a set amount of times and the best model is saved and how well it does is logged. The best of these runs will be saved and utilized for the final model. In the end, generation 17 performed the best with ~14000 lines cleared c:


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
from game import Game
import sys
import os
import joblib

def main():
    # textfile = open("./src/data/expectiminimax/10.csv", "w")
    # textfile.write("dropped, rows\n")
    # for i in range(2):
    #     print(i)
    #     g = Game(sys.argv[1])
    #     dropped, rows = g.run_no_visual()
    #     textfile.write(str(dropped) + ", " + str(rows) + "\n")
    # textfile.close()
    g = Game(sys.argv[1],joblib.load("genes/test1/g17.joblib"))
    g.run_no_visual()
    # g.run()


if __name__ == "__main__":
    main()


The above is the main.py file used to run the best model.